<a href="https://colab.research.google.com/github/tizbid/Datathon/blob/master/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.head()

### Data Preprocessing


In [ ]:
dataset.isna().sum()

Features and target variable

In [ ]:
X = dataset.drop(columns =['expenses']) #features
y = dataset.pop('expenses')             #labels


categorical  to numerical conversion

In [ ]:
#check  column cardinality 
features = [x for x in dataset.columns if x != 'expenses']
col_cardinality = {col:dataset[col].nunique() for col in features if dataset[col].dtype == "object"}
col_cardinality

one-hot encode feature dataset

In [ ]:
#Apply one-hot coding
X = pd.get_dummies(X, prefix='', prefix_sep='')

In [ ]:
print(X.shape)

Train Test Split

In [ ]:
train_dataset,test_dataset,train_labels,test_labels = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
train_dataset.head()

In [ ]:
print(train_dataset.shape,test_dataset.shape)

Normalize data

In [ ]:
def nom(x):
  stats = x.describe()
  stats = stats.transpose()
  return (x - stats['mean']) / stats['std']

In [ ]:
norm_train_dataset = nom(train_dataset)

In [ ]:
norm_train_dataset.tail()

In [ ]:
print(norm_train_dataset.shape,train_labels.shape)

### Model

Build model

In [ ]:
model= tf.keras.Sequential([
    layers.Dense(16,activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(32,activation=tf.nn.relu),
    layers.Dense(64,activation=tf.nn.relu),
    layers.Dense(1) ])


In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.0001),metrics= ['mae','mse'])

In [ ]:
model.summary()

fit model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(norm_train_dataset, train_labels, validation_split= 0.2,epochs=1000,callbacks=[callback],verbose = 0)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df

plot loss

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot(kind='line', ax=ax[0])
history_df[['mae', 'val_mae']].plot(kind='line', ax=ax[1]);

### Test Model

In [ ]:
test_dataset = nom(test_dataset)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
